## Домашнее задание по ВКонтакте

### Шаврина Татьяна

Нужно написать программу, которая скачивает комментарии из VK по VK API и кладёт их с помощью sqlite3 в базу данных. При этом база должна быть реляционной и многотабличной. Сохраняться при этом должны такие данные:
- id пользователя/сообщества, на стене которого находится комментарий
- id комментария
- id поста, к которому этот комментарий сделан
- id пользователя, который сделал этот комментарий
- id комментария, на который этот комментарий является ответом (если он действительно является ответом)
- время добавления комментария
- дата рождения автора комментария (если указана в профиле)
- город автора комментария (если указан в профиле)
- текст комментария

На гитхаб нужно положить код, а сам файл с базой данных -- куда-то в другое место (Яндекс.Диск, Google Drive и проч.) Ссылку на файл -- в README.md директории задания.

In [53]:
import sqlite3 
import json
import urllib.request
import requests
import re
import datetime


### Организация БД:

1. В одной таблице (USERS) хранится айди пользователя и вся информация о нем - дата рождения, город
2. Во второй таблице (WALLS) хранится айди всех сообществ и юзеров, со стены которых скачаны комментарии. В этой таблице у каждого уникального айди сообщества/юзера стоят в соответствии все айди постов стены, на которые скачаны комменты.
3. В третьей таблице (POSTS) лежат уникальные айди постов, и к ним все айди комментариев, которые к ним скачаны, и айди юзера/сообщества
4. В четвертой таблице (COMMENTS) - айди коммента, и к нему текст, время, айди поста, айди комментария-на-который-ответ, айди автора

In [64]:
#общение с БД

con = sqlite3.connect('vk3.db')
cur = con.cursor()

In [65]:
cur.execute("CREATE TABLE USERS (userid integer primarykey, bdate varchar, city varchar)")
cur.execute("CREATE TABLE WALLS (id integer primarykey, postids varchar)")
cur.execute("CREATE TABLE POSTS (postid integer primarykey, wallid integer, cids varchar)")
cur.execute("CREATE TABLE COMMENTS (cid integer primarykey, responseid integer, time varchar, userid integer, text varchar)")
con.commit()

In [66]:
def GetUser(userid):
    url = u'https://api.vk.com/method/users.get?'
    url = url + "user_ids=" +  str(userid) + "&fields=bdate,city"
    res = urllib.request.urlopen(url).read().decode('utf-8')
    info = json.loads(res)
    
    if 'bdate' in info['response'][0]:
        date = info['response'][0]['bdate']
        if len(date.split("."))<3:
            age = "None"
        else:
            if int(date.split(".")[1])<4:
                age = str(2017-int(date.split(".")[2]))
            else:
                age = str(2016-int(date.split(".")[2]))
        
    else:
        age = "None"
    city = str(info['response'][0]['city'])

    return[age,city]

In [67]:

def GetComments(ownerid, postid):
    lst = []
    url = u'https://api.vk.com/method/wall.getComments?'
    i=0
    while i<200:
        url = url + "owner_id=" + str(ownerid)+ "&post_id=" + str(postid) +"&offset=" + str(i) + "&count=100"
        res = urllib.request.urlopen(url).read().decode('utf-8')
        wall = json.loads(res)
        for j in range(1,100):
            try:
                commid = str(wall['response'][j]['cid'])
                text  = wall['response'][j]['text']
                if 'reply_to_cid' in wall['response'][j]:
                    response = str(wall['response'][j]['reply_to_cid'])
                else:
                    response = "None"
                authorid = str(wall['response'][j]['from_id'])
                time = datetime.datetime.fromtimestamp(wall['response'][j]['date']).strftime('%Y-%m-%d %H:%M:%S')
                age, city = GetUser(authorid)
                cur.execute('insert into users values (?, ?, ?)', (authorid,age,city ))
                cur.execute('insert into comments values  (?, ?, ?,?,?)', (commid ,response,time,authorid,text))
                con.commit()
                lst.append(commid)
                
            except:
                pass
        i+=100
    lst = ",".join(lst)
    #print(lst)
    return(lst)

In [68]:
def GetPosts(groupid):
    lst = []
    url = u'https://api.vk.com/method/wall.get?'
    i=0
    while i<200:

        url = url + "owner_id=" +str(groupid) + "&offset=" + str(i) + "&count=100"
        res = urllib.request.urlopen(url).read().decode('utf-8')
        wall = json.loads(res)
        for j in range(1,100):
            groupid = str(groupid)
            postid = str(wall['response'][j]['id'])            
            comments = wall['response'][j]['comments']['count']
            if comments>0:
                commentsdb=GetComments(groupid, postid)
            else:
                commentsdb="None"
            cur.execute('insert into posts values (?, ?, ?)', (postid ,groupid , commentsdb ))
            con.commit()
            lst.append(postid)
        i+=100
        lst = ",".join(lst)
        return(lst)

In [69]:
ids = [5350626,-34215577,8548609,-98745321,6275260]
for i in ids:
    posts = GetPosts(i)
    cur.execute('insert into walls values (?,?)', (str(i) ,posts))
    con.commit()

In [72]:
cur.execute("SELECT * FROM comments")
print(cur.fetchall())
con.commit()

[(6807, 'None', '2016-09-20 22:41:22', 101723410, 'поздравунькаю'), (6808, 'None', '2016-09-20 22:41:25', 5238907, 'Прекрасно и тянет на 18 *О*'), (6809, 6808, '2016-09-20 22:44:13', 5350626, '[id5238907|Лиза], ничего не меняется)'), (6810, 6809, '2016-09-20 22:45:02', 5238907, '[id5350626|Мария], ну так и хорошо, раз мгновение прекрасно)'), (6811, 'None', '2016-09-20 23:00:08', 256308182, 'встречаешь с котиками на юбке😻<br>такая красивая Маня😊'), (6812, 6811, '2016-09-20 23:00:51', 5350626, '[id256308182|Diana], меня окружают одни котики)'), (6813, 6812, '2016-09-20 23:01:15', 256308182, 'Мария, мяу❤️'), (6814, 'None', '2016-09-21 07:42:06', 249053117, 'С днём рождения!)'), (6816, 'None', '2016-09-21 08:42:15', 9264060, 'С днём рождения дорогая ! Крепкого здоровья , счастья и семейного уюта'), (6817, 'None', '2016-09-21 09:39:46', 2006047, 'И котиков!'), (6821, 'None', '2016-09-21 11:50:42', 3618, 'С Днем Рождения😍'), (6822, 'None', '2016-09-21 11:50:50', 3618, ''), (6825, 'None', '20

In [71]:
cur.execute("SELECT * FROM walls")
print(cur.fetchall())
con.commit()

[(5350626, '6806,6977,6976,6971,6969,6968,6967,6966,6965,6964,6963,6958,6955,6951,6949,6947,6946,6945,6944,6943,6942,6940,6939,6938,6935,6932,6931,6924,6919,6918,6912,6908,6904,6900,6895,6894,6890,6883,6879,6875,6874,6867,6866,6864,6857,6849,6848,6847,6845,6844,6843,6842,6840,6839,6838,6837,6834,6827,6824,6823,6819,6815,6802,6799,6792,6789,6788,6783,6782,6772,6765,6755,6741,6740,6739,6737,6728,6727,6726,6724,6723,6722,6721,6718,6717,6715,6708,6707,6706,6705,6704,6703,6702,6701,6700,6699,6698,6697,6695'), (-34215577, '931023,931018,931016,931013,931009,930991,930987,930981,930978,930969,930962,930960,930958,930948,930942,930937,930900,930898,930893,930877,930873,930869,930866,930861,930851,930833,930831,930830,930820,930815,930813,930810,930809,930796,930794,930788,930782,930776,930771,930768,930765,930761,930757,930754,930741,930734,930728,930721,930714,930710,930703,930699,930695,930692,930688,930687,930679,930670,930663,930620,930617,930612,930605,930600,930590,930585,930584,930579,9